# **Hugging Face Transformer Model For Text Summarization**
- Download **google/pegasus-cnn_dailymail** **transformer** based model from hugging face
- Predict **Text summarization** in the data Using pretrained model
- Do the **Fine tuning** of that pretrained model using our dataset **samsum**
- Do the Validation(Evalution) during Fine tuning using  - **Metric: ROUGE**
- Use the Fine tuned moddel and do the inferencing on new data to identify **Summarization** in that data
- **samsum** - SamSum Corpus: The SAMSum dataset contains about 16k messenger-like conversations with summaries.
- Here **google/pegasus-cnn_dailymail LLM model** used
- Here we will finetune the Pretrainined model by using bilsum data, which trying to make a summary based on the conversations


- Here **Pytourch /torch** method used, it will take care of all **compilation, optimizer** etc, no need to define adam compilation optimizer. In case of **Tensorflow**, we need to define it seperately

## Evaluation Metric:
- **"SacreBLEU"** is a metric commonly used for evaluating the quality of machine-generated translations, typically in the context of machine translation tasks. It is often used in NLP competitions and research to assess the performance of translation models.
- **Rouge (Recall-Oriented Understudy for Gisting Evaluation)** -  is a set of metrics used for evaluating the quality of summaries by comparing them to reference summaries. Rouge metrics are commonly used in natural language processing tasks, particularly in the evaluation of text summarization systems.

In [ ]:
print("ok!")

ok!


In [ ]:
!nvidia-smi

Wed Feb 28 13:50:30 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Downloading Libraries

In [ ]:
#!pip list  # Provides list of Libraries here

In [ ]:
"""
!pip install transformers[sentencepiece]  # This library converts entire corpus into Tokens
!pip install dataset  # Install Dataset Library
!pip install sacrebleu rouge_score  # Evaluation Libraries
!pip install py7zr  # 7zip Libraries to zip the file
"""
# Here install all libraries in single shot silently (-q)
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 48.4 MB/s eta 0:00:00


In [ ]:
# These accelerate libraries are related to GPU, To do utilize distributed process of the system/ parallele processing
!pip install --upgrade accelerate -q
!pip uninstall -y transformers accelerate -q  # Here uninstall transformers and resinstall again to get latest transformer
!pip install transformers accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 6.0 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm # Python library that provides a simple and convenient way to add progress bars to loops and iterable objects.
nltk.download("punkt")  # divides a text into a list of sentences

from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM  # Townload toenizer and se1seqLM model from transformer

from datasets import load_dataset,load_from_disk,load_metric

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
device="cuda" if torch.cuda.is_available() else "cpu"  # cuda is library supporting to gpu
device

'cuda'

## Downloading Hugging face **google/pegasus-cnn_dailymail** model

### 1. Use Pretrained Model - **google/pegasus-cnn_dailymail**'s Tokenizer
### 2. Load/download Pretrained Model - **google/pegasus-cnn_dailymail** itself

In [ ]:
model="google/pegasus-cnn_dailymail"

# Load the tokens from this model, we will get all weights to make the tokens
tokenizer=AutoTokenizer.from_pretrained(model)

#Load pretrained model to Cuda device
model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(model).to(device)#cuda


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

## Downloading Hugging face **samsum** data for Fine tune above model

- **samsum** is a SAMSum Corpus: A Human-annotated Dialogue Dataset for Abstractive Summarization.


In [ ]:
dataset=load_dataset("samsum")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [ ]:
print(dataset["train"]["dialogue"][0])

Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)


In [ ]:
dataset["train"]["summary"][0]

'Amanda baked cookies and will bring Jerry some tomorrow.'

In [ ]:
print(dataset["train"]["dialogue"][1])

Olivia: Who are you voting for in this election? 
Oliver: Liberals as always.
Olivia: Me too!!
Oliver: Great


In [ ]:
dataset["train"]["summary"][1]

'Olivia and Olivier are voting for liberals in this election. '

In [ ]:
for split in dataset:
  print(split)

train
test
validation


In [ ]:
split_lengths = [len(dataset[split])for split in dataset]
print(f"Length of each dataset- {split_lengths}")

Length of each dataset- [14732, 819, 818]


In [ ]:
for split in dataset:
  print(dataset[split])

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 819
})
Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 818
})


In [ ]:
dataset["train"].column_names

['id', 'dialogue', 'summary']

In [ ]:
print(dataset["test"]["dialogue"][0])

Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye


In [ ]:
print(dataset["test"]["summary"][0])

Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


## Apply this pretrained model tokenizer on  dataset to convert text to vector
- It converts Data/text to integer/number, based on pegasus Tokenizer dictonary/pretrained embedded model
- Once we apply tokenizer, it creats dictionary with **'input_ids' and 'attention_mask' and 'labels'** for that sentence

### Explanation of function - convert_examples_to_features:

-**Input Tokenization (input_encodings):**

- data_in_batch['dialogue']: This assumes that the input batch contains a key 'dialogue', representing the input dialogue text.

- tokenizer(..., max_length=1024, truncation=True): Tokenizes the input dialogue, **limiting the maximum length to 1024** tokens and truncating if necessary.

- input_encodings['input_ids']: Retrieves the input IDs from the tokenized input.

-**Target Tokenization (target_encodings):**

- with tokenizer.as_target_tokenizer():: This context manager allows you to use the tokenizer as a target tokenizer, which is typically used for labels in sequence-to-sequence tasks.

- tokenizer(..., max_length=128, truncation=True): Tokenizes the target summary, **limiting the maximum length to 128 tokens** and truncating if necessary.

- target_encodings['input_ids']: Retrieves the input IDs from the tokenized target summary.

-**Return Dictionary:**

- The function returns a dictionary containing 'input_ids', 'attention_mask', and 'labels'.

- **'input_ids' and 'attention_mask'** correspond to the **tokenized input dialogue.**

- **'labels'** corresponds to the **tokenized target summary.**

In [ ]:
#input_encoding=tokenizer(dataset["train"]["dialogue"],max_length=1024,truncation= True)
#input_encoding

In [ ]:
# This functions tokenizes(Converts to embeding) I/P feature --> dialogue
#and Output/Target feature --> summary also
# By default this function appends this info in existing data

def convert_examples_to_features(data_in_batch):

  #Apply Tokenizer to convert Input to vector
  input_encoding=tokenizer(data_in_batch["dialogue"],max_length=1024,truncation= True)

  #Apply Tokenizer to convert target to vector
  target_encoding=tokenizer(data_in_batch["summary"],max_length=128,truncation=True)

  return {
        'input_ids' : input_encoding['input_ids'],
        'attention_mask': input_encoding['attention_mask'],
        'labels': target_encoding['input_ids']
    }


In [ ]:
# Used convert_examples_to_features to convert word to embedding for all 3 data(Train/dtest/validation)
#Here we pass data in batch, which part of pytorch. It appends new features to existing data
dataset_en=dataset.map(convert_examples_to_features,batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
dataset["train"]

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [ ]:
dataset_en["train"]

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

> After this tokenizer, it adds these 3 features to data - **'input_ids', 'attention_mask', 'labels'**

In [ ]:
#1st lines encoded data
print(len(dataset_en["train"]["input_ids"][0]))
dataset_en["train"]["input_ids"][0]

In [ ]:
print(len(dataset_en["train"]["attention_mask"][0]))
dataset_en["train"]["attention_mask"][0]

In [ ]:
#1st lines Summary feature/Target features encoded data
print(len(dataset_en["train"]["labels"][0]))
dataset_en["train"]["labels"][0]

In [ ]:
dataset_en["train"]["attention_mask"][0]

### attention_mask meaning:
- **1:** Indicates that the corresponding token should be attended to (included in the computation of attention weights).

- **0:** Indicates that the corresponding token should be ignored (excluded from the computation of attention weights).

- For example, let's consider a sequence of tokens: ["Hello", ",", "how", "are", "you", "?"]. The attention mask could be: **[1,1,1,1,1,1]**

- In this case, all tokens are marked with 1, indicating that each token should be attended to during the self-attention process.

- However, in situations where padding is used to make sequences of equal length (common in batch processing), the attention mask may be adjusted to indicate which positions **correspond to actual tokens** and which **correspond to padding**. For example:
**[1,1,1,1,1,0]**

- Here, the last position is marked with 0, indicating that it corresponds to a padded token and should be ignored during attention.In the context of your code snippet, t

## Fine tune the model

In [ ]:
# RAG (will do it in the later session)
# finetuning(we are doing fine tuning over here)

from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq

In [ ]:
# https://huggingface.co/docs/transformers/v4.38.1/en/main_classes/trainer#transformers.Seq2SeqTrainingArguments

In [ ]:
model

In [ ]:
seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer,model=model)#model_pegasus

In [ ]:
training_args=TrainingArguments(
    output_dir='pegasus-samsum',    # This is fine tune model new name
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,  per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy='steps',    eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [ ]:
trainer=Trainer(model=model_pegasus, #Pretrained model initiated from AutoModelForSeq2SeqLM
                args=training_args, # All the hyper parameter
                tokenizer=tokenizer, # Define the tokenizer used
                data_collator=seq2seq_data_collator, # Define the tokenizer used
                train_dataset=dataset_en["test"], # Pass our preprocessed/tokenized train data. Here train data huge so passed test data
                eval_dataset=dataset_en["validation"])

In [ ]:
#Train the model / Fine tune the model
trainer.train()

## Evalation of model
- **rouge_metric** used evaluate

In [ ]:


def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,  #model_pegasus
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device),
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [ ]:
rouge_names=["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

In [ ]:
score = calculate_metric_on_test_ds(
    dataset['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

## Save the Fine tuned Model as **pegasus-samsum-model** and Tokenizer as **samsum-tokenizer** in local

In [ ]:
#Save the model
model_pegasus.save_pretrained("pegasus-samsum-model")

#Save the tokenizer
tokenizer.save_pretrained("samsum-tokenizer")

## Invoke/Call fine tuned new model - **pegasus-samsum-model** from Local along with Tokenizer

In [ ]:
#load Model+Tokenizer
tokenizer=AutoTokenizer.from_pretrained("/content/samsum-tokenizer")
model=AutoModelForSeq2SeqLM.from_pretrained("/content/pegasus-samsum-model").to('cuda')

In [ ]:
model

## Use **transformer pipeline** to do inference of fine tuned model

In [ ]:
sample_text_for_text=dataset["train"]["dialogue"][2]
print(sample_text_for_text)

In [ ]:
#Ground truth
summary=dataset["train"]["summary"][2]
summary

### Define model specific generating parameter like length penalty, max length etc

- This is specific to Pegasus model, so always check model specific document for such arguments

In [ ]:
gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

In [ ]:
#Create Pipeline with pipeline type as summarization and define loaded model and tokenizer
pipeline_object=pipeline("summarization",model=model,tokenizer=tokenizer)

In [ ]:
prediction=pipeline_object(sample_text_for_text)[0]["summary_text"]
print("Prdicted summary")
print(prediction)

In [ ]:
print("Ground truth summary")
summary

In [ ]:
https://huggingface.co/spaces/evaluate-metric/rouge

https://huggingface.co/docs/transformers/en/tasks/summarization

# END